## Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream [EEG+EDA+RESP_BELT] 

## Using Adaptive Random Forest 


- - New ensemble approach (Reward Penalty based Weighted Ensemble [RPWE])


## Date: 23 Sept 2021 at 10:10 AM

In [1]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from skmultiflow.meta import AdaptiveRandomForest
from window_slider import Slider

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):    
    list_values = list_values[0,:]
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [3]:
#======================================================
# EDA Feature Extraction (Wavelet Features)
#======================================================
def extract_eda_features(raw_eda):
    features =[]
    EDA = raw_eda
    list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features


In [4]:
#======================================================
# RESP BELT Feature Extraction (Wavelet Features)
#======================================================

def extract_resp_belt_features(raw_data):
    features =[]
    resp_belt = raw_data
    list_coeff = pywt.wavedec(resp_belt, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features

In [5]:
def eeg_features(raw_data):
    ch = 0
    features= []
    def calculate_entropy(list_values):
        counter_values = Counter(list_values).most_common()
        probabilities = [elem[1]/len(list_values) for elem in counter_values]
        entropy=scipy.stats.entropy(probabilities)
        return entropy

    def calculate_statistics(list_values):
        median = np.nanpercentile(list_values, 50)
        mean = np.nanmean(list_values)
        std = np.nanstd(list_values)
        var = np.nanvar(list_values)
        rms = np.nanmean(np.sqrt(list_values**2))
        return [median, mean, std, var, rms]

    def get_features(list_values):    
    #     list_values = list_values[0,:]
        entropy = calculate_entropy(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + statistics
    
    for i in range(raw_data.shape[0]):
        ch_data = raw_data[i]
        list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
        for coeff in list_coeff:
            features += get_features(coeff)
            
        ch = ch+1
    return features

In [6]:
# ##===================================================
# # EEG data read from files
# ##===================================================
# def eeg_data(p,v):
#     file_eeg = '/home/gp/Desktop/MER_arin/data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
#     print(file_eeg)
#     df = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
#     eeg_sig = df.loc[df.iloc[:,1] == v]
#     return eeg_sig

# ##===================================================
# # EDA data read from files
# ##===================================================
# def eda_data(p,v):
#     file_eda = '/home/gp/Desktop/MER_arin/data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
#     print(file_eda)
#     df = pd.read_csv(file_eda,sep=',', header = None, engine='python')
#     eda_sig = df.loc[df.iloc[:,1] == v]
#     return eda_sig

# ##===================================================
# # Resp data read from files
# ##===================================================
# def resp_data(p,v):
#     file_resp = '/home/gp/Desktop/MER_arin/data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
#     print(file_resp)
#     df = pd.read_csv(file_resp,sep=',', header = None, engine='python')
#     resp_sig = df.loc[df.iloc[:,1] == v]
#     return resp_sig

In [8]:
#================================================================
# Initialization of different parameters and performance metrics
#================================================================
segment_in_sec = 10 #in sec
bucket_size = int((8064/60)*segment_in_sec)  #8064 is for 60 sec record
overlap_count = 0

num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers

cBest_val = [] #Classifiers least error for valence classsification
cBest_aro = []  #Classifiers least error for arousal classsification
cWorst_val = [] #Classifiers heighest error for valence classsification
cWorst_aro = []  #Classifiers heighest error for arousal classsification

val_MSE = []
aro_MSE = []

beta_val = np.zeros(num_classifiers)#/num_classifiers
beta_aro = np.zeros (num_classifiers)#/num_classifiers

n_epoch =1
epochs =  1 #epoch is 1 because the model will be trained only once

optimizer= 'ARF' #optimizer
classifier = 'ARF_'+str(optimizer)
indx = 0
run = 1

participant = 32
videos = 40

global eeg_emotion, eda_emotion, resp_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
resp_emotion = []
mer_emotion = []

all_eta =[]
init_m = 0

#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()

eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()

resp_acc_val = metrics.Accuracy() #Accuracy
resp_f1m_val = metrics.F1() #F1 measure  
resp_acc_aro = metrics.Accuracy() #Accuracy
resp_f1m_aro = metrics.F1() #F1 measure
resp_roc_val = metrics.ROCAUC()
resp_roc_aro = metrics.ROCAUC()
resp_mcc_val = metrics.MCC()
resp_mcc_aro = metrics.MCC()

 
mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
resp_cm_val  = metrics.ConfusionMatrix()
resp_cm_aro  = metrics.ConfusionMatrix()
    
itr = 0 #controls the learning rate


for ii in range(0,participant):

        p =ii+1 #participant number
        
        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
        ##===================================================
        # EEG data read from files
        ##===================================================
        def eeg_data(p):
            file_eeg = '/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
            print(file_eeg)
            eeg_sig = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
            return eeg_sig

        ##===================================================
        # EDA data read from files
        ##===================================================
        def eda_data(p):
            file_eda = '/home/gp/Desktop/MER_arin/DEAP_data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
            print(file_eda)
            eda_sig = pd.read_csv(file_eda,sep=',', header = None, engine='python')
            return eda_sig

        ##===================================================
        # Resp data read from files
        ##===================================================
        def resp_data(p):
            file_resp = '/home/gp/Desktop/MER_arin/DEAP_data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
            print(file_resp)
            resp_sig = pd.read_csv(file_resp,sep=',', header = None, engine='python')
            return resp_sig
        
        #------------------------------------
        # Once file fetched data stored here 
        #------------------------------------
        grand_eeg = eeg_data(p)
        grand_eda = eda_data(p)
        grand_resp = resp_data(p)

        
        for jj in range(0,videos):
            v = jj+1 #Video number
            print('=========================================================================')
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            print(p_v)
            
            emotion_label =[]
            
            
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = grand_eeg.loc[grand_eeg.iloc[:,1] == v]
            eda_sig = grand_eda.loc[grand_eda.iloc[:,1] == v]
            resp_sig = grand_resp.loc[grand_resp.iloc[:,1] == v]
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            val = eeg_sig.iloc[0,8067]
            aro = eeg_sig.iloc[0,8068]
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al])
            
            #==========================================================
            # Predicted Valence and Arousal labels list initialization
            #==========================================================
            eeg_val_prdt=[]
            eeg_aro_prdt =[]
            
            eda_val_prdt=[]
            eda_aro_prdt =[]
            
            resp_val_prdt=[]
            resp_aro_prdt =[]
            
            
            #=========================================
            # Sliding window starts here 
            #=========================================
            slider_eeg = Slider(bucket_size,overlap_count)
            slider_eda = Slider(bucket_size,overlap_count)
            slider_resp = Slider(bucket_size,overlap_count)
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,32),range(3,8067)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(3,8067)]) #keeping only eda signals
            resp_sig = np.array(resp_sig.iloc[:,range(3,8067)]) #keeping only resp signals
            
            slider_eeg.fit(eeg_sig)
            slider_eda.fit(eda_sig)
            slider_resp.fit(resp_sig)

            while True:
                window_data_eeg = slider_eeg.slide()
                window_data_eda = slider_eda.slide() 
                window_data_resp = slider_resp.slide() 
                
                #=================================================
                # Feature extraction from EEG
                #=================================================
                features_eeg = eeg_features(window_data_eeg)
                x_eeg = np.array([features_eeg])  #EEG raw feature vector
#                 x_eeg = preprocessing.normalize(eeg) # EEG normalized features [0,1] 
                
                
                #=================================================
                # Feature extraction from EDA
                #=================================================
                eda_features = extract_eda_features(np.array(window_data_eda))
                x_eda = np.array([eda_features]) #EDA raw feature vector
#                 x_eda = preprocessing.normalize(eda) #EDA normalized features
                
                #=================================================
                # Feature extraction from Resp belt
                #=================================================

                resp_features = extract_resp_belt_features(np.array(window_data_resp))
                x_resp = np.array([resp_features]) #RESP BELT raw feature vector
#                 x_resp = preprocessing.normalize(resp) #RESP BELT normalized features
            
            
                #===================================================
                # Model initialization
                #===================================================
                if init_m == 0:
                    print('EEG Feature shape{}:'.format(x_eeg.shape))
                    print('EDA Feature shape{}:'.format(x_eda.shape))
                    print('RESP BELT Feature shape{}:'.format(x_resp.shape))

                    #========================
                    # For EEG data MLP model
                    #========================
                    eeg_model_val = AdaptiveRandomForest()
                    eeg_model_aro = AdaptiveRandomForest()

                    #========================
                    # For EDA data MLp model
                    #========================
                    eda_model_val = AdaptiveRandomForest()
                    eda_model_aro = AdaptiveRandomForest()
                    

                    #==============================
                    # For Resp Belt data MLP Model
                    #==============================
                    resp_model_val = AdaptiveRandomForest()
                    resp_model_aro = AdaptiveRandomForest()


                    init_m = init_m+1


                #===============================================================
                # Emotion Classification --> Valence and Arousal
                #===============================================================

                #===========================================
                # From EEG data -- RECS System
                #===========================================

                #Valence classification EEG

                #Test the model first 
                if (indx == 0): #When the model sees the first data
                    tmp_eeg_val = 0
                    y_pred_val_eeg = 2 #other than what are class labels
                else: 
                    eeg_val = eeg_model_val.predict_proba(x_eeg)
                    tmp_eeg_val = np.max(eeg_val)
                    y_pred_val_eeg = np.argmax(eeg_val)
                    
                #Train the model once
                eeg_model_val.partial_fit(x_eeg,y_act_val)

                eeg_acc_val = eeg_acc_val.update(y_act_val[0], y_pred_val_eeg)  # update the accuracy metric

                eeg_f1m_val = eeg_f1m_val.update(y_act_val[0], y_pred_val_eeg) #update f1 measure metric
                
                eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg)
                
                eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg)

                #Arousal classification EEG

                if (indx == 0): #When the model sees the first data
                    tmp_eeg_aro = 0
                    y_pred_aro_eeg = 2 #other than what are class labels
                else: 
                    eeg_aro = eeg_model_aro.predict_proba(x_eeg)
                    tmp_eeg_aro = np.max(eeg_aro)
                    y_pred_aro_eeg = np.argmax( eeg_aro)

                #Train the model once
                eeg_model_aro.partial_fit(x_eeg,y_act_aro)

                eeg_acc_aro = eeg_acc_aro.update(y_act_aro[0], y_pred_aro_eeg)  # update the accuracy metric

                eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro[0], y_pred_aro_eeg) #update f1 measure metric
                
                eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg)
                
                eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg)

                #===========================================
                # From EDA data 
                #===========================================

                #Valence classification EDA

                if (indx == 0): #When the model sees the first data
                    tmp_eda_val = 0
                    y_pred_val_eda = 2 #other than what are class labels
                else: 
                    eda_val = eda_model_val.predict_proba(x_eda) 
                    tmp_eda_val = np.max(eda_val)
                    y_pred_val_eda = np.argmax( eda_val)

                #Train the model once
                eda_model_val.partial_fit(x_eda,y_act_val)

                eda_acc_val = eda_acc_val.update(y_act_val[0], y_pred_val_eda)  # update the accuracy metric

                eda_f1m_val = eda_f1m_val.update(y_act_val[0], y_pred_val_eda) #update f1 measure metric
                
                eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda)
                
                eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda)
                

                #Arousal classification EDA
                if (indx == 0): #When the model sees the first data
                    tmp_eda_aro = 0
                    y_pred_aro_eda = 2 #other than what are class labels
                else: 
                    eda_aro = eda_model_aro.predict_proba(x_eda)
                    tmp_eda_aro = np.max(eda_aro)
                    y_pred_aro_eda = np.argmax(eda_aro)

                #Train the model once
                eda_model_aro.partial_fit(x_eda,y_act_aro)

                eda_acc_aro = eda_acc_aro.update(y_act_aro[0], y_pred_aro_eda)  # update the accuracy metric

                eda_f1m_aro = eda_f1m_aro.update(y_act_aro[0], y_pred_aro_eda) #update f1 measure metric
                
                eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda)
                
                eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda)

                #===========================================
                # From Resp Belt data
                #===========================================

                #Valence classification Resp Belt

                if (indx == 0): #When the model sees the first data
                    tmp_resp_val = 0
                    y_pred_val_resp = 2 #other than what are class labels
                else: 
                    resp_val  = resp_model_val.predict_proba(x_resp)
                    tmp_resp_val = np.max(resp_val)
                    y_pred_val_resp = np.argmax(resp_val)

                #Train the model once
                resp_model_val.partial_fit(x_resp,y_act_val)

                resp_acc_val = resp_acc_val.update(y_act_val[0], y_pred_val_resp)  # update the accuracy metric

                resp_f1m_val = resp_f1m_val.update(y_act_val[0], y_pred_val_resp) #update f1 measure metric
                
                resp_roc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp)
                
                resp_mcc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp)
                

                #Arousal classification Resp Belt
                if (indx == 0): #When the model sees the first data
                    tmp_resp_aro = 0
                    y_pred_aro_resp = 2 #other than what are class labels
                else: 
                    resp_aro = resp_model_aro.predict_proba(x_resp)
                    tmp_resp_aro = np.max(resp_aro)
                    y_pred_aro_resp = np.argmax(resp_aro)
               

                #Train the model once
                resp_model_aro.partial_fit(x_resp,y_act_aro)

                resp_acc_aro = resp_acc_aro.update(y_act_aro[0], y_pred_aro_resp)  # update the accuracy metric

                resp_f1m_aro = resp_f1m_aro.update(y_act_aro[0], y_pred_aro_resp) #update f1 measure metric
                
                resp_roc_aro = resp_roc_aro.update(y_act_aro[0], y_pred_aro_resp)
                
                resp_mcc_aro = resp_mcc_aro.update(y_act_aro[0], y_pred_aro_resp)
                
                
                indx = indx+1

                if slider_eeg.reached_end_of_list(): break
                
            #===================================================
            #   Error storing and comparing Part of each models
            #===================================================
            
            eeg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eeg_val))
            eeg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eeg_aro))
            
            eda_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eda_val))
            eda_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eda_aro))
            
            resp_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_resp_val))
            resp_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_resp_aro))
            
            val_MSE = [eeg_val_MSE, eda_val_MSE, resp_val_MSE] #Valence errors 
            aro_MSE = [eeg_aro_MSE, eda_aro_MSE, resp_aro_MSE] #Arousal errors
            

            
            #========================================================
            # Valence Arousal MSE Compariosn and storing in Cbest and Cworst
            #========================================================
            
            if (jj==0): #First Video
                cBest_val = val_MSE
                cBest_aro = aro_MSE
                
                cWorst_val  = val_MSE
                cWorst_aro = aro_MSE
                
            else:
                #-----------------------------------------
                #Beta for calence and arousal calculation
                #-----------------------------------------              
                beta_val = np.true_divide(list(np.subtract(cBest_val,val_MSE)), (1+np.exp(list(np.subtract(cWorst_val,cBest_val)))))
                beta_aro = np.true_divide(list(np.subtract(cBest_aro,aro_MSE)), (1+np.exp(list(np.subtract(cWorst_aro,cBest_aro)))))
                
                #--------------------------------------------------------------
                # Classifiers best MSE error and worst MSE error value update
                #-------------------------------------------------------------
                cBest_val = np.minimum(cBest_val, val_MSE)
                cBest_aro = np.minimum(cBest_aro, aro_MSE)
                cWorst_val = np.maximum(cWorst_val, val_MSE)
                cWorst_aro = np.maximum(cWorst_aro, aro_MSE)
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg)
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg)
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda)
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda)
            
            resp_cm_val = resp_cm_val.update(y_act_val[0], y_pred_val_resp)
            resp_cm_varo = resp_cm_aro.update(y_act_aro[0], y_pred_aro_resp)            
            
            #=============================================================
            # Storing All Results
            #=============================================================
            
            emotion_label.append([y_pred_val_eeg,y_pred_aro_eeg]) #appending valence & arousal predicted EEG
            emotion_label.append([y_pred_val_eda, y_pred_aro_eda]) #appending valence & arousal predicted EDA
            emotion_label.append([y_pred_val_resp, y_pred_aro_resp]) #appending valence & arousal predicted resp

            emotion_label = np.array(emotion_label)
            
                    
            #====================================================================
            # Decision label ensemble --> Reward Penalty Based Weightrd Ensemble
            #====================================================================
            val_label = emotion_label[:,0]
            aro_label = emotion_label[:,1]
            
            #------------------------------------------
            # Valence Class ensemble
            #------------------------------------------
            
            p_val = np.dot([w_val],val_label)
            y_prdt_mer_val = (p_val -0.5)
            if(y_prdt_mer_val > 0):
                mer_val = 1
            else:
                mer_val = 0

            #============================================
            # Weight update for Valence
            #============================================
            w_val = w_val + list(np.multiply(w_val,beta_val))    
            w_val_sum = sum(w_val) #total sum of weights
            w_val = np.array(w_val/w_val_sum) #weight rescaling


            
            #------------------------------------------
            # Arousal Class ensemble
            #------------------------------------------            
            p_val = np.dot([w_aro],aro_label)
            y_prdt_mer_aro = (p_val-0.5)
            if(y_prdt_mer_aro > 0):
                mer_aro = 1
            else:
                mer_aro = 0

            w_aro = w_aro + list(np.multiply(w_aro,beta_aro))
            w_aro_sum = sum(w_aro) #total sum of weights
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro = mer_cm_aro.update(y_act_aro[0], mer_aro)
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                         eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                         y_act_val[0], y_pred_val_eeg, y_act_aro[0], y_pred_aro_eeg]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                         eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                         y_act_val[0], y_pred_val_eda, y_act_aro[0], y_pred_aro_eda]))
            
            resp_emotion.append(np.array([p,v, resp_acc_val.get(), resp_f1m_val.get(), resp_roc_val.get(), resp_mcc_val.get(),
                                          resp_acc_aro.get(),resp_f1m_aro.get(), resp_roc_aro.get(), resp_mcc_aro.get(), 
                                          y_act_val[0], y_pred_val_resp, y_act_aro[0], y_pred_aro_resp]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                         mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                         y_act_val[0], mer_val, y_act_aro[0], mer_aro]))
            print('Actual emotion [valence: {}, arousal: {}]'.format(y_act_val, y_act_aro))
            print('Single modals Predicted emotion')
            print('[valence, arousal]:')
            print(emotion_label)
            
            print('MER Predicted emotion [valence: {}, arousal: {}]'.format(mer_val, mer_aro))
            print('Val MSE',val_MSE)
            print('Aro MSE',aro_MSE)
            # print('------------------------------')
            print('Beta_Val:', beta_val)
            print('Beta_Aro:', beta_aro)
            print('------------------------------')
            print('W Val:', w_val)
            print('W Aro:', w_aro)
            # print('------------------------------')
            # print('CBest Val MSE',cBest_val)
            # print('CBest Aro MSE',cBest_aro)
            # print('------------------------------')
            # print('Cworst Val MSE',cWorst_val)
            # print('Cworst Aro MSE',cWorst_aro)
            # print('==============================================')            
            # print('..................................................')
            # print('EEG')

            # print("Valence Accuracy:{}".format(round(eeg_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eeg_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eeg_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eeg_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eeg_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eeg_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eeg_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eeg_mcc_aro.get(),4)))

            # print('..................................................')
            # print('EDA')

            # print("Valence Accuracy:{}".format(round(eda_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eda_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eda_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eda_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eda_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eda_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eda_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eda_mcc_aro.get(),4)))


            # print('..................................................')
            # print('RESP BELT')

            # print("Valence Accuracy:{}".format(round(resp_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(resp_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(resp_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(resp_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(resp_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(resp_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(resp_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(resp_mcc_aro.get(),4)))

            # print('..................................................')
            print('MER model')

            print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            print('===============================xxxx================================')
                
                
                
            



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/1_data_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/eda_data/1_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/1_Respiration_data_from_DEAP.csv
Person: 1 Video:1
EEG Feature shape(1, 1152):
EDA Feature shape(1, 48):
RESP BELT Feature shape(1, 48):
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.0, 0.0, 0.0]
Aro MSE [0.0, 0.0, 0.0]
Beta_Val: [0. 0. 0.]
Beta_Aro: [0. 0. 0.]
------------------------------
W Val: [0.33333333 0.33333333 0.33333333]
W Aro: [0.33333333 0.33333333 0.33333333]
MER model
Valence Accuracy:1.0
Valence F1 score:1.0
Valence ROC score:-0.0
Valence MCC score:0.0
Arousal Accuracy:1.0
Arousal F1 score:1.0
Arousal ROC score:-0.0
Arousal MCC score:0.0
============================

Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.258066719446993, 0.1769361457210971, 0.3481066279114308]
Aro MSE [0.3338743528494674, 0.31049066301984635, 0.4229634954447981]
Beta_Val: [-0.11014055 -0.06703154 -0.14637218]
Beta_Aro: [-0.13308838 -0.11746309 -0.16429051]
------------------------------
W Val: [0.35107237 0.32849758 0.32043004]
W Aro: [0.39331407 0.28147834 0.32520758]
MER model
Valence Accuracy:0.8182
Valence F1 score:0.875
Valence ROC score:0.75
Valence MCC score:0.6236
Arousal Accuracy:0.9091
Arousal F1 score:0.8889
Arousal ROC score:0.9286
Arousal MCC score:0.8281
===============================xxxx================================
Person: 1 Video:12
Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [0 0]
 [1 0]]
MER Predicted emotion [valence: 1, arousal: 0]
Val MSE [0.14168825983360972, 0

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.07519110656074839, 0.045376436067167195, 0.012663116719206044]
Aro MSE [0.02195519926273427, 0.021578540320978855, 0.027171158700721756]
Beta_Val: [-0.03011256 -0.01717559 -0.00510731]
Beta_Aro: [-0.00868543 -0.00814981 -0.01055402]
------------------------------
W Val: [0.36332437 0.27394776 0.36272787]
W Aro: [0.41987586 0.27641954 0.3037046 ]
MER model
Valence Accuracy:0.7619
Valence F1 score:0.8
Valence ROC score:0.75
Valence MCC score:0.5095
Arousal Accuracy:0.8571
Arousal F1 score:0.8571
Arousal ROC score:0.8591
Arousal MCC score:0.7182
===============================xxxx================================
Person: 1 Video:22
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 0]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.055464886559

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.23354785533051697, 0.4967036517781139, 0.2062813885752845]
Aro MSE [0.04715257150807621, 0.12078376203761156, 0.038929996526240886]
Beta_Val: [-0.09353132 -0.18800901 -0.08228761]
Beta_Aro: [-0.01865345 -0.04561778 -0.01512147]
------------------------------
W Val: [0.37339116 0.26241912 0.36418972]
W Aro: [0.40867321 0.2684183  0.32290849]
MER model
Valence Accuracy:0.8065
Valence F1 score:0.8421
Valence ROC score:0.7906
Valence MCC score:0.5994
Arousal Accuracy:0.871
Arousal F1 score:0.875
Arousal ROC score:0.8708
Arousal MCC score:0.7417
===============================xxxx================================
Person: 1 Video:32
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.26642453538392

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/2_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/2_Respiration_data_from_DEAP.csv
Person: 2 Video:1
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.0039276727275896674, 0.010679676547560144, 0.005039441489581635]
Aro MSE [0.0015292160312807312, 0.0031866493654114326, 0.003651140958682904]
Beta_Val: [-0.00544645 -0.01320038 -0.01430313]
Beta_Aro: [-0.00090298 -0.00344276 -0.00082602]
------------------------------
W Val: [0.32444411 0.23932235 0.43623354]
W Aro: [0.42581786 0.25486448 0.31931766]
MER model
Valence Accuracy:0.8293
Valence F1 score:0.8293
Valence ROC score:0.8298
Valence MCC score:0.6595
Arousal Accuracy:0.9024
Arousal F1 score:0.92
Arousal ROC score:0.8975
Arousal MCC score:0.795
===============================xxxx================================
Person: 2 Video:2
Actual emotion 

...

Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.1281457032528717, 0.27645700399661477, 0.13987324840694668]
Aro MSE [0.22083719801472432, 0.19590714995415245, 0.12927429916123673]
Beta_Val: [-0.0533556  -0.11188767 -0.05985535]
Beta_Aro: [-0.08973178 -0.07729031 -0.04905827]
------------------------------
W Val: [0.38645332 0.06279889 0.55074779]
W Aro: [0.32192714 0.2932253  0.38484757]
MER model
Valence Accuracy:0.8406
Valence F1 score:0.859
Valence ROC score:0.8355
Valence MCC score:0.6815
Arousal Accuracy:0.8551
Arousal F1 score:0.8438
Arousal ROC score:0.8537
Arousal MCC score:0.709
===============================xxxx================================
Person: 4 Video:19
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.01974689374938

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.36082351331714685, 0.1365994520602011, 0.22621601038973602]
Aro MSE [0.0704815549149326, 0.10619094592979154, 0.02378502316971158]
Beta_Val: [-0.15266239 -0.05231868 -0.09486357]
Beta_Aro: [-0.02607926 -0.03671541 -0.008044  ]
------------------------------
W Val: [0.33240037 0.06687393 0.6007257 ]
W Aro: [0.29481036 0.27203067 0.43315897]
MER model
Valence Accuracy:0.8243
Valence F1 score:0.8434
Valence ROC score:0.8198
Valence MCC score:0.6483
Arousal Accuracy:0.8581
Arousal F1 score:0.8489
Arousal ROC score:0.8571
Arousal MCC score:0.7159
===============================xxxx================================
Person: 4 Video:29
Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.3728836721183

Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.4482241745815796, 0.46641408582137983, 0.3394856404961132]
Aro MSE [0.41859129549124435, 0.38400381556092633, 0.3397821400581962]
Beta_Val: [-0.17500899 -0.17917099 -0.13896167]
Beta_Aro: [-0.15836802 -0.14612862 -0.13718699]
------------------------------
W Val: [0.29403578 0.05643786 0.64952636]
W Aro: [0.27418202 0.29319111 0.43262687]
MER model
Valence Accuracy:0.8228
Valence F1 score:0.8333
Valence ROC score:0.8228
Valence MCC score:0.6508
Arousal Accuracy:0.8608
Arousal F1 score:0.8429
Arousal ROC score:0.8574
Arousal MCC score:0.719
===============================xxxx================================
Person: 4 Video:39
Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.331450814403016

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.0028551789119851114, 0.020311177179193028, 0.11859397563771182]
Aro MSE [0.05839356255903933, 0.007936835278770527, 0.04963432431701664]
Beta_Val: [ 0.00013085 -0.00782057 -0.04396864]
Beta_Aro: [-0.00528669  0.00131046 -0.01196631]
------------------------------
W Val: [0.29972344 0.05235463 0.64792193]
W Aro: [0.26836698 0.32830848 0.40332455]
MER model
Valence Accuracy:0.8144
Valence F1 score:0.8229
Valence ROC score:0.8141
Valence MCC score:0.6308
Arousal Accuracy:0.8563
Arousal F1 score:0.8356
Arousal ROC score:0.8519
Arousal MCC score:0.7102
===============================xxxx================================
Person: 5 Video:8
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.04176958

KeyboardInterrupt: 

In [8]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('RESP Val CM:')
print(resp_cm_val)
print('RESP Aro CM:')
print(resp_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    387    185
      1     62    646
EEG aro CM:
             0      1
      0    408    135
      1    109    628
EDA Val CM:
             0      1
      0    423    149
      1     70    638
EDA aro CM:
             0      1
      0    391    152
      1    100    637
RESP Val CM:
             0      1
      0    379    193
      1     53    655
RESP Aro CM:
             0      1
      0    372    171
      1     97    640
ReMECS Val CM:
             0      1
      0    383    189
      1     52    656
ReMECS aro CM:
             0      1
      0    404    139
      1     78    659


In [9]:
print('[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]')
#=============================
# EEG
#=============================
t_eeg_emotion =pd.DataFrame(eeg_emotion) 
t_eeg_emotion = t_eeg_emotion.iloc[:,range(2,10)]
t_eeg_emotion_avg = np.array(np.mean(t_eeg_emotion))
t_eeg_emotion_std = np.array(np.std(t_eeg_emotion))

print('EEG result Average:', t_eeg_emotion_avg)
print('EEG result std:', t_eeg_emotion_std)

#=============================
# EDA
#=============================
t_eda_emotion =pd.DataFrame(eda_emotion) 
t_eda_emotion = t_eda_emotion.iloc[:,range(2,10)]
t_eda_emotion_avg = np.array(np.mean(t_eda_emotion))
t_eda_emotion_std = np.array(np.std(t_eda_emotion))

print('EDA result Average:', t_eda_emotion_avg)
print('EDA result std:', t_eda_emotion_std)
#=============================
# RESP BELT
#=============================
t_resp_emotion =pd.DataFrame(resp_emotion) 
t_resp_emotion = t_resp_emotion.iloc[:,range(2,10)]
t_resp_emotion_avg = np.array(np.mean(t_resp_emotion))
t_resp_emotion_std = np.array(np.std(t_resp_emotion))

print('Resp Belt result Average:', t_resp_emotion_avg)
print('Resp Belt result std:', t_resp_emotion_std)
#=============================
# MER
#=============================
t_mer_emotion =pd.DataFrame(mer_emotion) 
t_mer_emotion = t_mer_emotion.iloc[:,range(2,10)]
t_mer_emotion_avg = np.array(np.mean(t_mer_emotion))
t_mer_emotion_std = np.array(np.std(t_mer_emotion))

print('MER result Average:', t_mer_emotion_avg)
print('MER result std:', t_mer_emotion_std)

[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]
EEG result Average: [0.74017122 0.78808365 0.72179428 0.47347638 0.71734764 0.73823267
 0.70890218 0.42279859]
EEG result std: [0.01948305 0.01394231 0.04472988 0.06265916 0.01853108 0.03861808
 0.0435912  0.04282887]
EDA result Average: [0.74450017 0.78225148 0.73230306 0.48081825 0.71974939 0.74416312
 0.70923477 0.42636305]
EDA result std: [0.02443615 0.02196177 0.04286983 0.05230158 0.02042    0.0318777
 0.04562099 0.04740045]
Resp Belt result Average: [0.72457194 0.76968808 0.70963179 0.70963179 0.68585031 0.70666886
 0.67511812 0.35753194]
Resp Belt result std: [0.01421573 0.01357758 0.03833005 0.03833005 0.02310075 0.05605706
 0.04178345 0.04169345]
MER result Average: [0.82569633 0.85239314 0.81315498 0.6480324  0.84008025 0.85121496
 0.83106238 0.67123735]
MER result std: [0.02890097 0.02229764 0.05021729 0.06318632 0.02115692 0.02732212
 0.05230778 0.05889766]


In [10]:
time_stamp = datetime.datetime.now()
run =1
algo= 'ARF/' 
filename = "/home/gp/Desktop/MER_arin/mer_codes/12DEAP_RPWE/Results_RPWE_DEAP/"+algo
fname_eeg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EDA'+'_' +classifier+'_results.csv'
fname_resp = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_RESP_BELT'+'_' +classifier+'_results.csv'
fname_mer = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
resp_emotion = pd.DataFrame(resp_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
resp_emotion.to_csv(fname_resp)
mer_emotion.to_csv(fname_mer)


